In [2]:
import pandas as pd
import numpy as np
import time
import datetime
import xgboost as xgb
from sklearn.linear_model import LassoCV, RidgeCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor

D:\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
from difflib import SequenceMatcher

In [4]:
from collections import Counter

In [5]:
from nltk.tokenize import word_tokenize

In [6]:
import re, math

In [7]:
import os

In [8]:
os.chdir("D:\IRDM")

In [9]:
df_all = pd.read_csv('df_all.csv', encoding="ISO-8859-1")

In [10]:
df_all.shape

(240760, 26)

In [11]:
def string_similarity(str1, str2):
    str1=str(str1)
    str2=str(str2)
    return SequenceMatcher(None, str1, str2).ratio()

def cosine_similarity(query, doc):
    query, doc = str(query), str(doc)
    WORD = re.compile(r'\w+')
    query, doc = Counter(WORD.findall(query)), Counter(WORD.findall(doc))
    intersection = set(query.keys()) & set(doc.keys())
    numerator = sum([query[x] * doc[x] for x in intersection])
    sum1 = sum([query[x]**2 for x in query.keys()])
    sum2 = sum([doc[x]**2 for x in doc.keys()])
    denominator = math.sqrt(sum1) * math.sqrt(sum2)
    if not denominator:
        return 0.0
    else:
        return float(numerator) / denominator

In [12]:
df_all['similarity_ratio_query_and_product_descrip'] = df_all.apply(lambda x: string_similarity(x['search_term'], x['product_description']), axis = 1)
df_all['similarity_ratio_query_and_product_title'] = df_all.apply(lambda x: string_similarity(x['search_term'], x['product_title']), axis = 1)
df_all['similarity_ratio_query_and_product_info'] = df_all.apply(lambda x: string_similarity(x['search_term'], x['product_info']), axis = 1)
df_all['similarity_ratio_query_and_product_attr'] = df_all.apply(lambda x: string_similarity(x['search_term'], x['attr']), axis = 1)
df_all['similarity_ratio_query_and_brand'] = df_all.apply(lambda x: string_similarity(x['search_term'], x['brand']), axis = 1)
print('5 features generated')

5 features generated


In [13]:
df_all['cosine_similarity_query_and_product_descrip'] = df_all.apply(lambda x: cosine_similarity(x['search_term'], x['product_description']), axis = 1)
df_all['cosine_similarity_query_and_product_title'] = df_all.apply(lambda x: cosine_similarity(x['search_term'], x['product_title']), axis = 1)
df_all['cosine_similarity_query_and_product_info'] = df_all.apply(lambda x: cosine_similarity(x['search_term'], x['product_info']), axis = 1)
df_all['cosine_similarity_query_and_product_attr'] = df_all.apply(lambda x: cosine_similarity(x['search_term'], x['attr']), axis = 1)
df_all['cosine_similarity_query_and_brand'] = df_all.apply(lambda x: cosine_similarity(x['search_term'], x['brand']), axis = 1)
print('5 features generated')

5 features generated


In [14]:
df_all.columns

Index(['Unnamed: 0', 'id', 'product_title', 'product_uid', 'relevance',
       'search_term', 'product_description', 'brand', 'product_info',
       'len_of_query', 'len_of_title', 'len_of_description', 'len_of_brand',
       'query_in_title', 'query_in_description', 'query_last_word_in_title',
       'query_last_word_in_description', 'word_in_title',
       'word_in_description', 'ratio_title', 'ratio_description', 'attr',
       'word_in_brand', 'ratio_brand', 'brand_feature', 'search_term_feature',
       'similarity_ratio_query_and_product_descrip',
       'similarity_ratio_query_and_product_title',
       'similarity_ratio_query_and_product_info',
       'similarity_ratio_query_and_product_attr',
       'similarity_ratio_query_and_brand',
       'cosine_similarity_query_and_product_descrip',
       'cosine_similarity_query_and_product_title',
       'cosine_similarity_query_and_product_info',
       'cosine_similarity_query_and_product_attr',
       'cosine_similarity_query_and_br

In [15]:
df_all.head()

,Unnamed: 0,id,product_title,product_uid,relevance,search_term,product_description,brand,product_info,len_of_query,...,similarity_ratio_query_and_product_descrip,similarity_ratio_query_and_product_title,similarity_ratio_query_and_product_info,similarity_ratio_query_and_product_attr,similarity_ratio_query_and_brand,cosine_similarity_query_and_product_descrip,cosine_similarity_query_and_product_title,cosine_similarity_query_and_product_info,cosine_similarity_query_and_product_attr,cosine_similarity_query_and_brand
0,0,2,simpson strong tie 12 gaug angl,100001,3.00,angl bracket,not onli do angl make joint stronger they also...,simpson strong tie,angl bracket\tsimpson strong tie 12 gaug angl\...,2,...,0.007712,0.186047,0.029162,0.558140,0.266667,0.138972,0.288675,0.259645,0.632456,0.0
1,1,3,simpson strong tie 12 gaug angl,100001,2.50,l bracket,not onli do angl make joint stronger they also...,simpson strong tie,l bracket\tsimpson strong tie 12 gaug angl\tno...,2,...,0.007742,0.050000,0.022032,0.486486,0.222222,0.000000,0.000000,0.087875,0.632456,0.0
2,2,9,behr premium textur deckov 1gal. #sc 141 tugbo...,100002,3.00,deck over,behr premium textur deckov is an innov solid c...,behr premium textur deckov,deck over\tbehr premium textur deckov 1gal. #s...,2,...,0.008239,0.177215,0.017110,0.400000,0.342857,0.108253,0.000000,0.166298,0.577350,0.0
3,3,16,delta vero 1 handl shower onli faucet trim kit...,100005,2.33,rain shower head,updat your bathroom with the delta vero singl ...,delta,rain shower head\tdelta vero 1 handl shower on...,3,...,0.000000,0.326087,0.043302,0.842105,0.095238,0.039103,0.154303,0.172825,0.866025,0.0
4,4,17,delta vero 1 handl shower onli faucet trim kit...,100005,2.67,shower onli faucet,updat your bathroom with the delta vero singl ...,delta,shower onli faucet\tdelta vero 1 handl shower ...,3,...,0.000000,0.382979,0.048452,0.857143,0.260870,0.117309,0.462910,0.306719,0.866025,0.0


In [16]:
df_all.shape

(240760, 36)

In [17]:
def normalisation(df):
    df_norm = (df - df.min()) / (df.max() - df.min())
    return df_norm

In [18]:
def create_result(prediction, info):
    result = pd.DataFrame()
    result['id'] = df_all['id'][num_train:].astype(str)
    prediction[prediction > 3.0] = 3.0
    prediction[prediction < 1.0 ] = 1.0
    result['relevance'] = prediction
    result.to_csv(info, index = False)

In [19]:
df_all_copy = df_all.copy()

In [20]:
df_all.drop(['Unnamed: 0', 'attr',], axis = 1, inplace = True)

In [21]:
df_all.columns

Index(['id', 'product_title', 'product_uid', 'relevance', 'search_term',
       'product_description', 'brand', 'product_info', 'len_of_query',
       'len_of_title', 'len_of_description', 'len_of_brand', 'query_in_title',
       'query_in_description', 'query_last_word_in_title',
       'query_last_word_in_description', 'word_in_title',
       'word_in_description', 'ratio_title', 'ratio_description',
       'word_in_brand', 'ratio_brand', 'brand_feature', 'search_term_feature',
       'similarity_ratio_query_and_product_descrip',
       'similarity_ratio_query_and_product_title',
       'similarity_ratio_query_and_product_info',
       'similarity_ratio_query_and_product_attr',
       'similarity_ratio_query_and_brand',
       'cosine_similarity_query_and_product_descrip',
       'cosine_similarity_query_and_product_title',
       'cosine_similarity_query_and_product_info',
       'cosine_similarity_query_and_product_attr',
       'cosine_similarity_query_and_brand'],
      dtype='ob

In [22]:
df_all.head()

,id,product_title,product_uid,relevance,search_term,product_description,brand,product_info,len_of_query,len_of_title,...,similarity_ratio_query_and_product_descrip,similarity_ratio_query_and_product_title,similarity_ratio_query_and_product_info,similarity_ratio_query_and_product_attr,similarity_ratio_query_and_brand,cosine_similarity_query_and_product_descrip,cosine_similarity_query_and_product_title,cosine_similarity_query_and_product_info,cosine_similarity_query_and_product_attr,cosine_similarity_query_and_brand
0,2,simpson strong tie 12 gaug angl,100001,3.00,angl bracket,not onli do angl make joint stronger they also...,simpson strong tie,angl bracket\tsimpson strong tie 12 gaug angl\...,2,6,...,0.007712,0.186047,0.029162,0.558140,0.266667,0.138972,0.288675,0.259645,0.632456,0.0
1,3,simpson strong tie 12 gaug angl,100001,2.50,l bracket,not onli do angl make joint stronger they also...,simpson strong tie,l bracket\tsimpson strong tie 12 gaug angl\tno...,2,6,...,0.007742,0.050000,0.022032,0.486486,0.222222,0.000000,0.000000,0.087875,0.632456,0.0
2,9,behr premium textur deckov 1gal. #sc 141 tugbo...,100002,3.00,deck over,behr premium textur deckov is an innov solid c...,behr premium textur deckov,deck over\tbehr premium textur deckov 1gal. #s...,2,12,...,0.008239,0.177215,0.017110,0.400000,0.342857,0.108253,0.000000,0.166298,0.577350,0.0
3,16,delta vero 1 handl shower onli faucet trim kit...,100005,2.33,rain shower head,updat your bathroom with the delta vero singl ...,delta,rain shower head\tdelta vero 1 handl shower on...,3,14,...,0.000000,0.326087,0.043302,0.842105,0.095238,0.039103,0.154303,0.172825,0.866025,0.0
4,17,delta vero 1 handl shower onli faucet trim kit...,100005,2.67,shower onli faucet,updat your bathroom with the delta vero singl ...,delta,shower onli faucet\tdelta vero 1 handl shower ...,3,14,...,0.000000,0.382979,0.048452,0.857143,0.260870,0.117309,0.462910,0.306719,0.866025,0.0


In [23]:
df_train = pd.read_csv('train.csv', encoding="ISO-8859-1")

In [24]:
num_train = df_train.shape[0]
train_x = df_all.iloc[:num_train,8:]
test_x  = df_all.iloc[num_train:,8:]

In [25]:
# Normalisation of features before learning
train_x = normalisation(train_x).values
train_y = df_all['relevance'][:num_train].values
test_x = normalisation(test_x).values

In [26]:
print(train_x.shape)
print(train_y.shape)
pd.DataFrame(train_x).head(5)

(74067, 26)
(74067,)


,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,0.090909,0.178571,0.130536,0.4,0.0,0.0,0.0,0.0,0.090909,0.090909,...,0.026135,0.186047,0.107810,0.486123,0.266667,0.178302,0.288675,0.287122,0.409126,0.0
1,0.090909,0.178571,0.130536,0.4,0.0,0.0,0.0,0.0,0.090909,0.090909,...,0.026237,0.050000,0.079683,0.397146,0.222222,0.000000,0.000000,0.086785,0.409126,0.0
2,0.090909,0.392857,0.170163,0.6,0.0,0.0,0.0,0.0,0.090909,0.090909,...,0.027921,0.177215,0.060268,0.289749,0.342857,0.138889,0.000000,0.178250,0.320538,0.0
3,0.181818,0.464286,0.100233,0.0,0.0,0.0,0.0,0.0,0.090909,0.090909,...,0.000000,0.326087,0.163593,0.838746,0.095238,0.050169,0.154303,0.185863,0.784619,0.0
4,0.181818,0.464286,0.100233,0.0,0.5,0.0,1.0,1.0,0.272727,0.272727,...,0.000000,0.382979,0.183912,0.857419,0.260870,0.150508,0.462910,0.342024,0.784619,0.0


In [27]:
print(test_x.shape)
pd.DataFrame(test_x).head(5)

(166693, 26)


,0,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,0.090909,0.172414,0.146448,0.4,0.000000,0.000000,0.0,0.0,0.000000,0.000000,...,0.030594,0.177778,0.097341,0.565919,0.062500,0.000000,0.000000,0.092439,0.409126,0.000000
1,0.181818,0.172414,0.146448,0.4,0.000000,0.000000,0.0,0.0,0.090909,0.090909,...,0.018333,0.173913,0.104197,0.584957,0.121212,0.000000,0.000000,0.116202,0.529137,0.000000
2,0.181818,0.172414,0.146448,0.4,0.000000,0.000000,0.0,0.0,0.090909,0.090909,...,0.012222,0.565217,0.104197,0.584957,0.606061,0.051270,0.235702,0.201621,0.704995,0.333333
3,0.181818,0.172414,0.146448,0.4,0.111111,0.004673,1.0,1.0,0.272727,0.272727,...,0.000000,0.734694,0.124568,0.633763,1.000000,0.205078,0.707107,0.409697,1.000000,1.000000
4,0.363636,0.172414,0.146448,0.4,0.000000,0.000000,0.0,0.0,0.272727,0.272727,...,0.000000,0.701754,0.177485,0.723586,0.818182,0.158853,0.547723,0.378373,0.929559,0.774597


In [28]:
df_all.to_csv('df_all_all.csv')

### Models

In [31]:
# Xgboost model
model_xgb = xgb.XGBRegressor(n_estimators=500, max_depth=10, learning_rate=0.1).fit(train_x, train_y)
test_y_xgb = model_xgb.predict(test_x)
create_result(test_y_xgb, 'xgboost.csv')

In [32]:
# Random forest model
model_rf = RandomForestRegressor(n_estimators=10).fit(train_x, train_y)
test_y_rf = model_rf.predict(test_x)
create_result(test_y_rf, 'rand_forest.csv')

In [33]:
# Gradient boost model
model_grad_boost = GradientBoostingRegressor(random_state=0, loss='ls').fit(train_x, train_y)
test_y_grad_boost = model_grad_boost.predict(test_x)
create_result(test_y_grad_boost, 'grad_boost.csv')

In [34]:
# KNN model
model_knn = KNeighborsRegressor().fit(train_x, train_y)
test_y_knn = model_knn.predict(test_x)
create_result(test_y_knn, 'knn.csv')

In [35]:
# SVM model
model_svm = SVR(C=1, epsilon=0.2).fit(train_x, train_y)
test_y_svm = model_svm.predict(test_x)
create_result(test_y_svm, 'svm.csv')